# Training

In [1]:
import sys
sys.path.append('/vol/tensusers5/nhollain/s_clip_scripts')

from itertools import chain
from main import main
from params import parse_args

fashion = False

if fashion:
    str_args = '''--train-data Fashion-ALL
            --label-ratio 0.1
            --val-data Fashion-ALL
            --keyword-path keywords/fashion/class-name.txt
            --epochs 10
            --method base  
    '''
else:
    str_args = ''' --train-data RS-ALL
            --label-ratio 0.1
            --val-data RS-ALL
            --imagenet-val RSICD-CLS \
            --keyword-path keywords/RS/class-name.txt
            --epochs 25
            --zeroshot-frequency 5  
            --method base
            --active-learning
    '''
str_args = str_args.split('\n')
str_args = [s.strip() for s in str_args]
str_args = [s.split(' ') for s in str_args]
str_args = list(chain(*str_args))
str_args = [s for s in str_args if len(s) > 0]
args = parse_args(str_args)

In [ ]:
%%time
checkpoint_path = main(args) # Calls the main.py function of S-CLIP

Cannot parse args
Making log path: ./checkpoint/2023_10_30-10_09_34-data_RS-ALL-ratio_0.1-model_RN50-method_base-keyword_none-AL_True-PL_method_None-vit_False
Running with a single process on device cuda.
Getting data...
RS-ALL
Split: val
Data size: 1094
Data size: 210
Data size: 58
RSICD-CLS
Split: val
RS-ALL
Split: train
Data size: 8734
Data size: 1680
Data size: 497
Dataset size: 10911


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2752/2752 [01:38<00:00, 27.96it/s]


Labeled size: 43
Data got.
Start epoch 0
Done training.
Done evaluating.
RS-ALL
Split: val
Data size: 1094
Data size: 210
Data size: 58
RSICD-CLS
Split: val
RS-ALL
Split: train
Data size: 8734
Data size: 1680
Data size: 497
Dataset size: 10911


 26%|█████████████████████████████████████▌                                                                                                             | 704/2752 [00:22<01:05, 31.44it/s]

# Eval

In [ ]:
%%time
checkpoint = checkpoint_path 
if 'Fashion' in checkpoint:
    zeroshot_datasets = ["Fashion200k-SUBCLS", "Fashion200k-CLS", "FashionGen-CLS", "FashionGen-SUBCLS", "Polyvore-CLS", ]
    retrieval_datasets = ["FashionGen", "Polyvore", "Fashion200k",]
else:
    zeroshot_datasets = ["RSICD-CLS", "UCM-CLS"] # "WHU-RS19", "RSSCN7", "AID" -> NOT WORKING bc of different data-loading workings
    retrieval_datasets = ["RSICD", "UCM", "Sydney"]

for dataset in zeroshot_datasets:
    str_args = ['--name', checkpoint, '--imagenet-val', dataset]
    args = parse_args(str_args)
    main(args)

for dataset in retrieval_datasets:
    str_args = ['--name', checkpoint, '--val-data', dataset]
    args = parse_args(str_args)
    main(args)